## Associating significant L1000FWD drug signatures with respective drugs
### Database : http://amp.pharm.mssm.edu/L1000FWD/

In [1]:
import pandas as pd
import os
import csv
from itertools import islice
from collections import defaultdict
import json

In [2]:
os.chdir('../../scripts')
from export_script import *
from gene_resolver import *
os.chdir('../notebooks/L1000FWD')

In [3]:
df_sig_metadata = pd.read_csv('input/CD_signature_metadata.csv')
df_sig_metadata['pert_desc'] = df_sig_metadata['pert_desc'].str.lower()

In [4]:
df_sig_metadata.head()

,sig_id,SCS_centered_by_batch,batch,cell_id,mean_cosine_dist_centered_by_batch,pert_desc,pert_dose,pert_id,pert_time
0,AML001_CD34_6H:BRD-K43389675:10,0.0,AML001_CD34_6H,CD34,0.853471,daunorubicin,10.00000,BRD-K43389675,6.0
1,AML001_PC3_6H:BRD-A19037878:0.37037,0.0,AML001_PC3_6H,PC3,0.505995,trichostatin_a,0.37037,BRD-A19037878,6.0
2,AML001_PC3_6H:BRD-A19037878:1.11111,0.0,AML001_PC3_6H,PC3,0.676204,trichostatin_a,1.11111,BRD-A19037878,6.0
3,AML001_PC3_6H:BRD-A19037878:10,0.0,AML001_PC3_6H,PC3,0.747633,trichostatin_a,10.00000,BRD-A19037878,6.0
4,AML001_PC3_6H:BRD-A19037878:3.33333,0.0,AML001_PC3_6H,PC3,0.659851,trichostatin_a,3.33333,BRD-A19037878,6.0


In [5]:
# Creating separate files for up and down genes

with open('input/CD_signatures_binary_42809.gmt', 'r') as fin, open('input/L1000FWD_signatures_up.gmt', 'w') as fout:
    fout.writelines(islice(fin, 0, None, 2))
fout.close()

with open('input/CD_signatures_binary_42809.gmt', 'r') as fin, open('input/L1000FWD_signatures_down.gmt','w') as fout:
    fout.writelines(islice(fin, 1, None, 2))
fout.close()

### Filtering df_sig_metadata by lowest mean_cosine_dist_centered_by_batch

In [6]:
# Get the smallest mean cosine distance for each pert id #
df_filtered = df_sig_metadata.sort_values(by = ['pert_id', 'mean_cosine_dist_centered_by_batch'])\
    .groupby('pert_id')\
    .head(1)

In [7]:
len(df_filtered)

4941

In [8]:
df_filtered.head()

,sig_id,SCS_centered_by_batch,batch,cell_id,mean_cosine_dist_centered_by_batch,pert_desc,pert_dose,pert_id,pert_time
26739,CPC015_PHH_24H:BRD-A00100033-001-04-8:10,0.0203,CPC015_PHH_24H,PHH,0.911062,nifurtimox,10.0,BRD-A00100033,24.0
336,CPC001_HA1E_24H:BRD-A00267231-001-01-1:10,0.0022,CPC001_HA1E_24H,HA1E,0.897247,hemado,10.0,BRD-A00267231,24.0
31189,CPC019_MCF7_24H:BRD-A00420644-001-01-7:10,0.0000,CPC019_MCF7_24H,MCF7,0.728397,-666,10.0,BRD-A00420644,24.0
16182,CPC010_HA1E_6H:BRD-A00474148-001-02-3:10,0.0007,CPC010_HA1E_6H,HA1E,0.913214,-666,10.0,BRD-A00474148,6.0
17487,CPC011_HA1E_6H:BRD-A00520476-001-05-8:10,0.0147,CPC011_HA1E_6H,HA1E,0.890211,"6h-pyrido[2,3-b][1,4]benzodiazepin-6-one, 11-[...",10.0,BRD-A00520476,6.0


### Importing L1000FWD drug metadata to match pert_ids

In [9]:
df_metadata = pd.read_csv('input/Drugs_metadata.csv', usecols = ['pert_iname','pert_id','pubchem_cid',
                                                                'LSM_id','inchi_key'])
df_metadata.dropna(subset = ['pubchem_cid'], inplace = True)
df_metadata['pubchem_cid'] = df_metadata['pubchem_cid'].astype(int)
df_metadata['inchi_key'] = df_metadata['inchi_key'].astype(str)
df_metadata['inchi_key'] = df_metadata['inchi_key'].apply(lambda x: str(x[9:]))

In [10]:
df_metadata.head()

,pert_id,LSM_id,pert_iname,inchi_key,pubchem_cid
0,BRD-A00100033,LSM-1232,nifurtimox,ARFHIAQFJWUCFH-UHFFFAOYSA-N,6842999
1,BRD-A00520476,LSM-22625,otenzepad,UBRKDAVQCKZSPO-UHFFFAOYSA-N,107867
2,BRD-A00546892,LSM-1235,biperiden,YSXKPIUOCJLQIE-UHFFFAOYSA-N,92151
3,BRD-A00758722,LSM-1237,noretynodrel,ICTXHFFSOAJUMG-OQPPHWFISA-N,5702095
4,BRD-A00827783,LSM-4299,dyphylline,KSCFJBIXMNOVSH-UHFFFAOYSA-N,3182


In [11]:
df_main = df_filtered.merge(df_metadata)

In [12]:
df_main.head()

,sig_id,SCS_centered_by_batch,batch,cell_id,mean_cosine_dist_centered_by_batch,pert_desc,pert_dose,pert_id,pert_time,LSM_id,pert_iname,inchi_key,pubchem_cid
0,CPC015_PHH_24H:BRD-A00100033-001-04-8:10,0.0203,CPC015_PHH_24H,PHH,0.911062,nifurtimox,10.0,BRD-A00100033,24.0,LSM-1232,nifurtimox,ARFHIAQFJWUCFH-UHFFFAOYSA-N,6842999
1,CPC001_HA1E_24H:BRD-A00267231-001-01-1:10,0.0022,CPC001_HA1E_24H,HA1E,0.897247,hemado,10.0,BRD-A00267231,24.0,LSM-1233,hemado,KOCIMZNSNPOGOP-UHFFFAOYSA-N,4043357
2,CPC019_MCF7_24H:BRD-A00420644-001-01-7:10,0.0000,CPC019_MCF7_24H,MCF7,0.728397,-666,10.0,BRD-A00420644,24.0,LSM-6366,SA-3676,ASCBUEVCEVGOFP-UHFFFAOYSA-N,2853908
3,CPC010_HA1E_6H:BRD-A00474148-001-02-3:10,0.0007,CPC010_HA1E_6H,HA1E,0.913214,-666,10.0,BRD-A00474148,6.0,LSM-1234,BRD-A00474148,RCGAUPRLRFZAMS-UHFFFAOYSA-N,44825297
4,CPC011_HA1E_6H:BRD-A00520476-001-05-8:10,0.0147,CPC011_HA1E_6H,HA1E,0.890211,"6h-pyrido[2,3-b][1,4]benzodiazepin-6-one, 11-[...",10.0,BRD-A00520476,6.0,LSM-22625,otenzepad,UBRKDAVQCKZSPO-UHFFFAOYSA-N,107867


### Importing Drugbank mapping file specific to LINCS small molecule identifiers

In [13]:
drugbank_mapping = pd.read_csv('../../metadata/drugmonizome_metadata.tsv', sep = '\t', usecols = ['DrugBank ID',
                                                                                                 'InChI Key'])
drugbank_mapping = drugbank_mapping.rename(columns = {'InChI Key':'inchi_key'})

In [14]:
drugbank_mapping.head()

,DrugBank ID,inchi_key
0,DB00006,OIRCOABEOLEUMC-GEJPAHFPSA-N
1,DB00007,GFIJNRVAKGFPGQ-LIJARHBVSA-N
2,DB00014,BLCLNMBMMGCOAS-URPVMXJPSA-N
3,DB00027,NDAYQJDHGXTBJL-MWWSRJDJSA-N
4,DB00035,NFLWUMRGJYTJIN-PNIOQBSNSA-N


In [15]:
df_drugbank = df_main.merge(drugbank_mapping)

1000 small molecules found in Drugbank, however we need to include the L1000 drugs for analysis

In [16]:
len(set(df_drugbank['DrugBank ID']))

1000

In [17]:
df_drugbank.head()

,sig_id,SCS_centered_by_batch,batch,cell_id,mean_cosine_dist_centered_by_batch,pert_desc,pert_dose,pert_id,pert_time,LSM_id,pert_iname,inchi_key,pubchem_cid,DrugBank ID
0,CPC015_A375_6H:BRD-A00546892-001-01-8:10,0.0413,CPC015_A375_6H,A375,0.948594,-666,10.0,BRD-A00546892,6.0,LSM-1235,biperiden,YSXKPIUOCJLQIE-UHFFFAOYSA-N,92151,DB00810
1,CPC016_MCF7_24H:BRD-A00993607-003-15-4:10,0.0021,CPC016_MCF7_24H,MCF7,0.921066,-666,10.0,BRD-A00993607,24.0,LSM-1238,alprenolol,PAZJSJFMUHDSTF-UHFFFAOYSA-N,66368,DB00866
2,CPC009_VCAP_6H:BRD-A01320529-001-08-3:10,0.0000,CPC009_VCAP_6H,VCAP,0.618531,salmeterol,10.0,BRD-A01320529,6.0,LSM-1240,salmeterol,GIIZNNXWQWCKIB-UHFFFAOYSA-N,5152,DB00938
3,CPD003_PC3_24H:BRD-A01787639-300-06-0:10,0.0220,CPD003_PC3_24H,PC3,0.903126,naftopidil dihydrochloride,10.0,BRD-A01787639,24.0,LSM-4302,naftopidil,HRRBJVNMSRJFHQ-UHFFFAOYSA-N,4418,DB12092
4,CPC010_HEPG2_6H:BRD-A01826957-093-13-7:10,0.0056,CPC010_HEPG2_6H,HEPG2,0.921420,xanthinol,10.0,BRD-A01826957,6.0,LSM-4950,xanthinol,DSFGXPJYDCSWTA-UHFFFAOYSA-N,9912,DB09092


Creating a L1000FWD specific metadata table for InChI Keys not found in Drugbank

In [18]:
l1000_metadata = df_main
for index,row in l1000_metadata.iterrows():
    drugbank_list = df_drugbank['inchi_key'].tolist()
    if row['inchi_key'] in drugbank_list:
        l1000_metadata.drop(index, inplace = True) # drop drugbank drugs

In [19]:
len(l1000_metadata)

3898

In [20]:
l1000_metadata.head()

,sig_id,SCS_centered_by_batch,batch,cell_id,mean_cosine_dist_centered_by_batch,pert_desc,pert_dose,pert_id,pert_time,LSM_id,pert_iname,inchi_key,pubchem_cid
0,CPC015_PHH_24H:BRD-A00100033-001-04-8:10,0.0203,CPC015_PHH_24H,PHH,0.911062,nifurtimox,10.0,BRD-A00100033,24.0,LSM-1232,nifurtimox,ARFHIAQFJWUCFH-UHFFFAOYSA-N,6842999
1,CPC001_HA1E_24H:BRD-A00267231-001-01-1:10,0.0022,CPC001_HA1E_24H,HA1E,0.897247,hemado,10.0,BRD-A00267231,24.0,LSM-1233,hemado,KOCIMZNSNPOGOP-UHFFFAOYSA-N,4043357
2,CPC019_MCF7_24H:BRD-A00420644-001-01-7:10,0.0000,CPC019_MCF7_24H,MCF7,0.728397,-666,10.0,BRD-A00420644,24.0,LSM-6366,SA-3676,ASCBUEVCEVGOFP-UHFFFAOYSA-N,2853908
3,CPC010_HA1E_6H:BRD-A00474148-001-02-3:10,0.0007,CPC010_HA1E_6H,HA1E,0.913214,-666,10.0,BRD-A00474148,6.0,LSM-1234,BRD-A00474148,RCGAUPRLRFZAMS-UHFFFAOYSA-N,44825297
4,CPC011_HA1E_6H:BRD-A00520476-001-05-8:10,0.0147,CPC011_HA1E_6H,HA1E,0.890211,"6h-pyrido[2,3-b][1,4]benzodiazepin-6-one, 11-[...",10.0,BRD-A00520476,6.0,LSM-22625,otenzepad,UBRKDAVQCKZSPO-UHFFFAOYSA-N,107867


In [21]:
# Create specific output dataframe to concatenate to master drugominzome metadata table
l1000_drugmonizome_metadata = l1000_metadata.rename(columns = {'pert_id':'Accession Numbers', 'pert_iname':'Common name',
                                                 'inchi_key':'InChI Key'})

In [22]:
# Exporting to metadata directory
l1000_drugmonizome_metadata.to_csv('../../metadata/l1000fwd_metadata.tsv', sep = '\t', columns = ['Common name','InChI Key',
                                                                                    'Accession Numbers'], index = False)

### Concatenate the two dataframes back together and create ouput files

In [23]:
df_output = df_drugbank.append(l1000_metadata, ignore_index=True, sort = False)

In [24]:
df_output.head()

,sig_id,SCS_centered_by_batch,batch,cell_id,mean_cosine_dist_centered_by_batch,pert_desc,pert_dose,pert_id,pert_time,LSM_id,pert_iname,inchi_key,pubchem_cid,DrugBank ID
0,CPC015_A375_6H:BRD-A00546892-001-01-8:10,0.0413,CPC015_A375_6H,A375,0.948594,-666,10.0,BRD-A00546892,6.0,LSM-1235,biperiden,YSXKPIUOCJLQIE-UHFFFAOYSA-N,92151,DB00810
1,CPC016_MCF7_24H:BRD-A00993607-003-15-4:10,0.0021,CPC016_MCF7_24H,MCF7,0.921066,-666,10.0,BRD-A00993607,24.0,LSM-1238,alprenolol,PAZJSJFMUHDSTF-UHFFFAOYSA-N,66368,DB00866
2,CPC009_VCAP_6H:BRD-A01320529-001-08-3:10,0.0000,CPC009_VCAP_6H,VCAP,0.618531,salmeterol,10.0,BRD-A01320529,6.0,LSM-1240,salmeterol,GIIZNNXWQWCKIB-UHFFFAOYSA-N,5152,DB00938
3,CPD003_PC3_24H:BRD-A01787639-300-06-0:10,0.0220,CPD003_PC3_24H,PC3,0.903126,naftopidil dihydrochloride,10.0,BRD-A01787639,24.0,LSM-4302,naftopidil,HRRBJVNMSRJFHQ-UHFFFAOYSA-N,4418,DB12092
4,CPC010_HEPG2_6H:BRD-A01826957-093-13-7:10,0.0056,CPC010_HEPG2_6H,HEPG2,0.921420,xanthinol,10.0,BRD-A01826957,6.0,LSM-4950,xanthinol,DSFGXPJYDCSWTA-UHFFFAOYSA-N,9912,DB09092


In [25]:
# Drop duplicate perturbation IDs
df_output.drop_duplicates(subset = ['pert_id'], inplace = True)

In [26]:
# Create lookup of signature IDs matched to pert_ids
pert_id_dict = df_output.set_index('sig_id').to_dict()['pert_id']

In [27]:
# Creating up and downregulated genelists for significant signature IDs
with open('input/L1000FWD_signatures_up.gmt', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    d_up = {pert_id_dict.get(line[0]): # map signature ID to pert_id
            ([(str(g)).split(',')[0]
            for g in line[2:]])
            for line in reader if line[0] in df_output['sig_id'].tolist()}
    
with open('input/L1000FWD_signatures_down.gmt', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    d_down = {pert_id_dict.get(line[0]): # map signature ID to pert_id
              ([(str(g)).split(',')[0] 
                for g in line[2:]])
                for line in reader if line[0] in df_output['sig_id'].tolist()}

### Exporting a dictionary of pert ids matched to InChI Keys for querying through SEP-L1000 API and Enrichr

In [28]:
pert_id_inchi = df_output[['pert_id','inchi_key']].copy()

In [29]:
pert_id_inchi.head()

,pert_id,inchi_key
0,BRD-A00546892,YSXKPIUOCJLQIE-UHFFFAOYSA-N
1,BRD-A00993607,PAZJSJFMUHDSTF-UHFFFAOYSA-N
2,BRD-A01320529,GIIZNNXWQWCKIB-UHFFFAOYSA-N
3,BRD-A01787639,HRRBJVNMSRJFHQ-UHFFFAOYSA-N
4,BRD-A01826957,DSFGXPJYDCSWTA-UHFFFAOYSA-N


In [30]:
pert_id_inchi.to_csv('input/pert_id_inchi.tsv', sep = '\t', index = False)

### Exporting GMT file of drugs matched to signatures for Enrichr API Querying

In [31]:
gmt_formatter(d_up, 'input/L1000FWD_enrichr_query_up.txt')
gmt_formatter(d_down, 'input/L1000FWD_enrichr_query_down.txt')

### Exporting drug-set libraries in GMT format

In [32]:
# Transposing dictionary so that genes are set-labels and drugs are set members
drugsetlibrary_up = transposer(d_up)
drugsetlibrary_down = transposer(d_down)

In [33]:
def resolve_drugsetlibrary(library):
    gene_list = []
    df_genes = pd.DataFrame()

    for gene, drugs in library.items():
        gene_list.append(gene)
    df_genes['Gene Name'] = gene_list
    
    gene_resolver(df_genes) # script that will match synonyms to approved symbols
    gene_dict = df_genes.set_index('Gene Name').to_dict()['Approved Symbol']
    approved_genes = df_genes['Approved Symbol'].tolist() # separate comparison list of approved symbols
    
    # Create dictionary of pert IDs matched to InChI Keys
    inchi_lookup = pert_id_inchi.set_index('pert_id').to_dict()['inchi_key']
    
    # Get each approved symbol for synonyms
    drugsetlibrary = {gene_dict.get(k,k):v for k,v in library.items()}
    
    # Filter out any unapproved symbols
    drugsetlibrary = {k: drugsetlibrary[k] for k in approved_genes if k in drugsetlibrary}
    
    # Map pert_id to inchikey
    drugsetlibrary = {k:[inchi_lookup.get(x,x) for x in v] for k,v in drugsetlibrary.items()}
    
    # Drop duplicates and sets with less than 5 drugs
    drugsetlibrary = {k:list(set(v)) for k,v in drugsetlibrary.items() if len(set(v)) >=5}
    
    return drugsetlibrary

In [34]:
drugsetlibrary_upregulated = resolve_drugsetlibrary(drugsetlibrary_up)

drugsetlibrary_downregulated = resolve_drugsetlibrary(drugsetlibrary_down)

In [35]:
os.chdir('../../data/L1000FWD')

gmt_formatter(drugsetlibrary_upregulated, 'L1000FWD_signature_drugsetlibrary_up.gmt')

gmt_formatter(drugsetlibrary_downregulated, 'L1000FWD_signature_drugsetlibrary_down.gmt')

### Library counts

In [36]:
library_counts(drugsetlibrary_upregulated)

4884 unique drugs
7611 unique association terms
1087474 unique associations
142.88188148732098 average drugs per term


In [37]:
library_counts(drugsetlibrary_downregulated)

4884 unique drugs
7622 unique association terms
1060251 unique associations
139.10404093413803 average drugs per term
